In [ ]:
import os
import time
import math
import random
import numpy as np
import pandas as pd
from google import genai
from google.genai import types
from dotenv import load_dotenv

In [ ]:
random.seed(42)
load_dotenv()
client = genai.Client(api_key=os.getenv("GEMINI_API_KEY"))
prompt_template = ''

In [3]:
def build_prompt(text):
    return prompt_template.replace('KALIMAT:', text)

In [4]:
def load_data():
    data = pd.read_csv('./val_data.csv', encoding='latin-1')
    return data

In [ ]:
safety_settings = [
    types.SafetySetting(
        category=types.HarmCategory.HARM_CATEGORY_HATE_SPEECH,
        threshold=types.HarmBlockThreshold.BLOCK_NONE,
    ),
    types.SafetySetting(
        category=types.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT,
        threshold=types.HarmBlockThreshold.BLOCK_NONE,
    ),
    types.SafetySetting(
        category=types.HarmCategory.HARM_CATEGORY_HARASSMENT,
        threshold=types.HarmBlockThreshold.BLOCK_NONE,
    ),
    types.SafetySetting(
        category=types.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT,
        threshold=types.HarmBlockThreshold.BLOCK_NONE,
    ),
    types.SafetySetting(
        category=types.HarmCategory.HARM_CATEGORY_CIVIC_INTEGRITY,
        threshold=types.HarmBlockThreshold.BLOCK_NONE,
    ),
]

def generate_noise(prompt):
    response = client.models.generate_content(
        model="gemini-2.5-flash", 
        contents=prompt, 
        config=types.GenerateContentConfig(
            safety_settings=safety_settings,
        )
    )
    return response.text

# CHAR LEVEL NOISE

In [6]:
# Typos
prompt_template = f'''KALIMAT:

- lakukan noise injection dengan mensimulasikan typo pada kalimat tersebut.
- lakukan pada maksimal 25% dari kata yang ada pada kalimat tersebut.
- output hanya berupa kalimat akhir yang telah diubah
- jangan tambahkan markdown dan tanda baca apapun
- gunakan huruf kecil'''

In [7]:
data = load_data()
data.head()

,review,ac,air_panas,bau,general,kebersihan,linen,service,sunrise_meal,tv,wifi
0,over all baik tapi kalau bisa kolong tempat ti...,neut,neut,neut,pos,neg,neut,neut,neut,neut,neut
1,"Ac harus dicek secara Berkala, tanpa harus men...",neg,neut,neut,neut,neut,neut,neut,neut,neut,neut
2,Nyaman tp ac bocor,neg,neut,neut,neut,neut,neut,neut,neut,neut,neut
3,Kamarnya kotor apalagi kamar mandi kotor tak t...,neut,neut,neut,neut,neg,neut,neut,neut,neut,neg
4,agak kotor di kamar yg lantai 2.,neut,neut,neut,neut,neg,neut,neut,neut,neut,neut


In [10]:
random.seed(42)
augmented_cnt = 0
for i in range(len(data)):
    n = random.uniform(0, 1)

    if n <= 0.1:
        augmented_cnt += 1
        text = data.iloc[i]['review']

        print("Index:", i)
        print("Original:", text, '\n')

        augmented = generate_noise(build_prompt(text))
        data["review"].values[i] = augmented

        print("Augmented:", augmented)
        print("=" * 50)
        time.sleep(4)

Index: 1
Original: ac harus dixcek secara berkalak tanpa harus menunggu customer complained terimakasih 

Augmented: ac jarus dixcek secara berkakak tanpa harus menunggu customer complained terimakasih
Index: 7
Original: bagus nyamn sayang waktu ak nginpe kamar mandi nya bau bgt semoga lebih baik lg kdepan rekomended bgt 

Augmented: bagusss nyamn sayang watku ak nginpe kmar mandi nya bau bgt semuga lebih baik lg kdepan rekomended bgt
Index: 9
Original: kamar nyaman staf ramah cuman wif error mulu pesen with brekfast sampe jam 10 pagi blm dantar masa harus diingeti dulu lampu kamar agak remang2 

Augmented: kmr nyman syaf ramah cuman wif error mlulu pesen with brekfast sampe jam 10 paig blm dantar masa harus diingeti dlu lampu kamar agak remang2
Index: 12
Original: biasa aaja remmote ac ga berfungii baru kali ini saya kecewa dgan airy biasanya dapet souvernir dari airy yg ini boro boro welcom snack aja ga ada 

Augmented: biasa ajaa remoote ac ga berfungi baru kali ini saya kevewa dagn

In [11]:
print("Total noisy data generated:", augmented_cnt)

Total noisy data generated: 56


In [12]:
data.to_csv('./noisy/typo.csv', index=False)

# SYNONYM REPLACEMENT

In [ ]:
prompt_template = f'''KALIMAT:

- lakukan noise injection dengan mengganti maksimum 25% kata dalam kalimat dengan sinonim nya
- output hanya berupa 1 baris kalimat akhir yang telah diubah
- jangan tambahkan markdown dan tanda baca apapun
- jangan hilangkan kata selain dari kata yang diganti
- gunakan huruf kecil'''

In [ ]:
data = load_data()
data.head()

,review,ac,air_panas,bau,general,kebersihan,linen,service,sunrise_meal,tv,wifi
0,over all baik tapi kalau bisa kolong tempat ti...,neut,neut,neut,pos,neg,neut,neut,neut,neut,neut
1,"Ac harus dicek secara Berkala, tanpa harus men...",neg,neut,neut,neut,neut,neut,neut,neut,neut,neut
2,Nyaman tp ac bocor,neg,neut,neut,neut,neut,neut,neut,neut,neut,neut
3,Kamarnya kotor apalagi kamar mandi kotor tak t...,neut,neut,neut,neut,neg,neut,neut,neut,neut,neg
4,agak kotor di kamar yg lantai 2.,neut,neut,neut,neut,neg,neut,neut,neut,neut,neut


In [ ]:
random.seed(42)
augmented_cnt = 0
for i in range(len(data)):
    n = random.uniform(0, 1)

    if n <= 0.1:
        augmented_cnt += 1
        text = data.iloc[i]['review']

        print("Index:", i)
        print("Original:", text, '\n')

        augmented = generate_noise(build_prompt(text))
        data["review"].values[i] = augmented

        print("Augmented:", augmented)
        print("=" * 50)
        time.sleep(4)

Index: 1
Original: Ac harus dicek secara Berkala, tanpa harus menunggu customer Complained, terimakasih  

Augmented: ac wajib diperiksa secara berkala tanpa harus menunggu customer complained terimakasih
Index: 7
Original: Bagus nyaman,  sayang waktu ak nginep kamar mandi nya bau bgt,  semoga lebih baik lg kedepan,  Recomended bgt 

Augmented: oke enak, sayang saat ak nginep kamar mandi nya bau sangat, semoga lebih baik lg kedepan, recomended bgt
Index: 9
Original: kamar nyaman, staff ramah. cuman wifi error mulu, pesen with breakfast sampe jam 10 pagi blm diantar, masa harus diingetin dulu. lampu kamar agak remang2 

Augmented: kamar enak karyawan ramah hanya wifi masalah terus pesen with breakfast sampe jam 10 pagi blm diantar masa harus diingatkan dulu lampu kamar agak remang2
Index: 12
Original: Biasa aja. Remote ac ga berfungsi. Baru kali ini saya kecewa dgn airy. Biasanya dapet souvenir dari airy, yg ini boro boro, welcome snack aja ga ada  

Augmented: biasa saja remote ac tida

In [ ]:
print("Total noisy data generated:", augmented_cnt)

Total noisy data generated: 56


In [ ]:
data.to_csv('./noisy/synonym-replacement.csv', index=False)

# WORD INSERTION

In [6]:
prompt_template = f'''KALIMAT:

- lakukan noise injection dengan menyisipkan kata-kata irrelevan yang membuat kalimat menjadi tidak nyambung
- sebar 3-6 kata tambahan di seluruh kalimat
- jangan mengubah kata yang sudah ada, hanya sisipkan kata baru
- output hanya berupa 1 baris kalimat akhir yang telah diubah
- jangan tambahkan markdown dan tanda baca apapun
- jangan hilangkan kata selain dari kata yang diganti
- gunakan huruf kecil'''

In [7]:
data = load_data()
data.head()

,review,ac,air_panas,bau,general,kebersihan,linen,service,sunrise_meal,tv,wifi
0,over all baik tapi kalau bisa kolong tempat ti...,neut,neut,neut,pos,neg,neut,neut,neut,neut,neut
1,"Ac harus dicek secara Berkala, tanpa harus men...",neg,neut,neut,neut,neut,neut,neut,neut,neut,neut
2,Nyaman tp ac bocor,neg,neut,neut,neut,neut,neut,neut,neut,neut,neut
3,Kamarnya kotor apalagi kamar mandi kotor tak t...,neut,neut,neut,neut,neg,neut,neut,neut,neut,neg
4,agak kotor di kamar yg lantai 2.,neut,neut,neut,neut,neg,neut,neut,neut,neut,neut


In [8]:
random.seed(42)
augmented_cnt = 0
for i in range(len(data)):
    n = random.uniform(0, 1)

    if n <= 0.1:
        augmented_cnt += 1
        text = data.iloc[i]['review']

        print("Index:", i)
        print("Original:", text, '\n')

        augmented = generate_noise(build_prompt(text))
        data["review"].values[i] = augmented

        print("Augmented:", augmented)
        print("=" * 50)

        time.sleep(4)

Index: 1
Original: Ac harus dicek secara Berkala, tanpa harus menunggu customer Complained, terimakasih  

Augmented: ac kucing harus dicek meja secara biru berkala cepat tanpa harus menunggu customer kemarin complained terimakasih
Index: 7
Original: Bagus nyaman,  sayang waktu ak nginep kamar mandi nya bau bgt,  semoga lebih baik lg kedepan,  Recomended bgt 

Augmented: bagus apel nyaman sayang waktu awan ak nginep kamar meja mandi nya bau buku bgt semoga lebih baik lg kedepan recomended bgt
Index: 9
Original: kamar nyaman, staff ramah. cuman wifi error mulu, pesen with breakfast sampe jam 10 pagi blm diantar, masa harus diingetin dulu. lampu kamar agak remang2 

Augmented: kamar nyaman pisang staff ramah cuman wifi error mulu kursi pesen with breakfast sampe jam 10 pagi blm diantar langit masa harus diingetin dulu biru lampu kamar mobil agak remang2
Index: 12
Original: Biasa aja. Remote ac ga berfungsi. Baru kali ini saya kecewa dgn airy. Biasanya dapet souvenir dari airy, yg ini bor

In [9]:
print("Total noisy data generated:", augmented_cnt)

Total noisy data generated: 56


In [10]:
data.to_csv('./noisy/word-insertion.csv', index=False)

# WORD DELETION

In [6]:
data = load_data()
data.head()

,review,ac,air_panas,bau,general,kebersihan,linen,service,sunrise_meal,tv,wifi
0,over all baik tapi kalau bisa kolong tempat ti...,neut,neut,neut,pos,neg,neut,neut,neut,neut,neut
1,"Ac harus dicek secara Berkala, tanpa harus men...",neg,neut,neut,neut,neut,neut,neut,neut,neut,neut
2,Nyaman tp ac bocor,neg,neut,neut,neut,neut,neut,neut,neut,neut,neut
3,Kamarnya kotor apalagi kamar mandi kotor tak t...,neut,neut,neut,neut,neg,neut,neut,neut,neut,neg
4,agak kotor di kamar yg lantai 2.,neut,neut,neut,neut,neg,neut,neut,neut,neut,neut


In [7]:
random.seed(42)
augmented_cnt = 0
for i in range(len(data)):
    n = random.uniform(0, 1)

    if n <= 0.1:
        augmented_cnt += 1
        text = data.iloc[i]['review']
        words = text.split()
        num_words = len(words)
        num_to_del = math.ceil(len(text.split(' ')) / 4)

        print("Index:", i)
        print("Original:", text, '\n')

        if num_to_del > 0:
            indices_to_delete = np.random.choice(num_words, size=num_to_del, replace=False)
            mask = np.ones(num_words, dtype=bool)
            mask[indices_to_delete] = False
            augmented_words = np.array(words)[mask]
            augmented = ' '.join(augmented_words)
        else:
            augmented = text  # if too short, don't change

        data.at[i, "review"] = augmented

        print("Augmented:", augmented)
        print("=" * 50)
        # time.sleep(5)

Index: 1
Original: Ac harus dicek secara Berkala, tanpa harus menunggu customer Complained, terimakasih  

Augmented: Ac harus dicek Berkala, tanpa harus menunggu customer
Index: 7
Original: Bagus nyaman,  sayang waktu ak nginep kamar mandi nya bau bgt,  semoga lebih baik lg kedepan,  Recomended bgt 

Augmented: Bagus nyaman, waktu ak nginep kamar nya bau semoga lebih kedepan, Recomended
Index: 9
Original: kamar nyaman, staff ramah. cuman wifi error mulu, pesen with breakfast sampe jam 10 pagi blm diantar, masa harus diingetin dulu. lampu kamar agak remang2 

Augmented: kamar nyaman, staff ramah. error mulu, pesen with sampe jam blm diantar, masa harus lampu kamar agak remang2
Index: 12
Original: Biasa aja. Remote ac ga berfungsi. Baru kali ini saya kecewa dgn airy. Biasanya dapet souvenir dari airy, yg ini boro boro, welcome snack aja ga ada  

Augmented: aja. ac berfungsi. kali ini saya kecewa dgn airy. Biasanya dapet dari airy, ini boro boro, welcome snack ga ada
Index: 19
Original:

In [8]:
print("Total noisy data generated:", augmented_cnt)

Total noisy data generated: 56


In [9]:
data.to_csv('./noisy/word-deletion.csv', index=False)

# COMBINED

In [6]:
prompt_template = f'''KALIMAT:

- lakukan 3 noise injection: typo, penghapusan kata, atau penyisipan kata-kata irrelevan yang membuat kalimat menjadi tidak nyambung
- lakukan noise injection pada maksimum 40% kata dari kalimat
- pastikan kalimat akhir memiliki masing-masing macam noise injection
- jika ada beberapa kata yang sama, hanya ubah satu kata saja
- jangan tambahkan markdown dan tanda baca apapun
- gunakan huruf kecil
- output hanya berupa kalimat akhir yang telah diubah
'''

In [7]:
data = load_data()
data.head()

,review,ac,air_panas,bau,general,kebersihan,linen,service,sunrise_meal,tv,wifi
0,over all baik tapi kalau bisa kolong tempat ti...,neut,neut,neut,pos,neg,neut,neut,neut,neut,neut
1,"Ac harus dicek secara Berkala, tanpa harus men...",neg,neut,neut,neut,neut,neut,neut,neut,neut,neut
2,Nyaman tp ac bocor,neg,neut,neut,neut,neut,neut,neut,neut,neut,neut
3,Kamarnya kotor apalagi kamar mandi kotor tak t...,neut,neut,neut,neut,neg,neut,neut,neut,neut,neg
4,agak kotor di kamar yg lantai 2.,neut,neut,neut,neut,neg,neut,neut,neut,neut,neut


In [8]:
random.seed(42)
augmented_cnt = 0

for i in range(len(data)):
    n = random.uniform(0, 1)

    if n <= 0.1:
        augmented_cnt += 1
        text = data.iloc[i]['review']

        print("Index:", i)
        print("Original:", text, '\n')

        augmented = generate_noise(build_prompt(text))
        data["review"].values[i] = augmented

        print("Augmented:", augmented)
        print("=" * 50)

        time.sleep(4)

Index: 1
Original: Ac harus dicek secara Berkala, tanpa harus menunggu customer Complained, terimakasih  

Augmented: ac harus dicek lalu secaraa berkala tanpa menunggu customer complained terimakasih
Index: 7
Original: Bagus nyaman,  sayang waktu ak nginep kamar mandi nya bau bgt,  semoga lebih baik lg kedepan,  Recomended bgt 

Augmented: bagus nayman sayang bola ak nginep kamar mandi nya bau bgt semoga lebih baik lg kedepan recomended bgt
Index: 9
Original: kamar nyaman, staff ramah. cuman wifi error mulu, pesen with breakfast sampe jam 10 pagi blm diantar, masa harus diingetin dulu. lampu kamar agak remang2 

Augmented: kamar nyamn staff ramah wifi error mulu kucing pesen with breakfast sampe jam 10 pagi blm diantar masa harus diingetin dulu lampu kamar agak remang2
Index: 12
Original: Biasa aja. Remote ac ga berfungsi. Baru kali ini saya kecewa dgn airy. Biasanya dapet souvenir dari airy, yg ini boro boro, welcome snack aja ga ada  

Augmented: bisaa aja remote jalan ga berfungsi 

In [9]:
print("Total noisy data generated:", augmented_cnt)

Total noisy data generated: 56


In [10]:
data.to_csv('./noisy/combined-noise.csv', index=False)